In [ ]:
from typing import Any
import gradio as gr
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

load_dotenv(override=True)


inst=f"""You are a Clarifying agent for a Mutual fund selector agent, Given a user's investment-related query, ask exactly 3 clarifying questions:, to help the mutual fund selector select the best funds,
      you ask a list of 3 questions, the questions should be about 1. Tenure 2. Systematic investment plan amount 3. the fund type- whether it is small cap, large cap, mid cap or nifty """



class clarifying_model(BaseModel):
    questions: list[str]=Field(description="the list of clarifying questions to ask the user")
Clarifying_agent=Agent(
    name="Clarifying Agent",
    instructions=inst,
    model="gpt-4o-mini",
    output_type=clarifying_model
)
class QAInput(BaseModel):
    tenure: str
    sip: str
    fund_expectation: str
class mf_top_5(BaseModel):
    top_5: list[str]=Field(description="It is the output model that gives out top 5 mutual funds based on input")

ins=""" You are a mutual fund list generator.

You MUST return the result in a list of string pattern:
use the answers provided as input to you to get the tenure and sip amount

Use the populate_model tool to infer fund category.
Pick 5 Indian mutual funds based on tenure, SIP and expected CAGR.
DO NOT add explanations."""
Mf_list=Agent[Any](
    name="mutual fund list generator",
    model="gpt-4o-mini",
    output_type=mf_top_5)

class MFSelectorInput(BaseModel):
    tenure: str
    sip: str
    fund_expectation: str

mf_list_generator = Mf_list.as_tool(
    tool_name="mutual_fund_list_generator",
    tool_description="Generates top 5 mutual funds based on tenure, SIP and expected CAGR"
)
instr = """
You are a mutual fund selector.

You will be given:
- tenure
- monthly SIP
- expected CAGR

Step 1:Call the tool `mutual_fund_list_generator` with these values. 
Step 2: From the returned top 5 funds, choose ONE best fund.
Step 3: Write a 300-word report covering:
- the top 5 funds you intially had, the one fund that you choose,
- its Past performance
- Future outlook
- Expected return over the given tenure

Be clear and professional."""

mf_selector=Agent(
    name="Mutual fund selector and report creator",
    instructions=instr,
    tools=[mf_list_generator],
    model="gpt-4o-mini",
    output_type=str
)
async def Run_Clarifier(Query: str):
    with trace("Clarifying agent"):
        result = await Runner.run(Clarifying_agent, Query)
        #print(result.final_output)
        quest="\n".join([f"{i+1} : {q}" for i, q in enumerate(result.final_output.questions)])
        return(quest,gr.update(visible=True),gr.update(visible=True),gr.update(visible=True),gr.update(visible=True))#a1,a2,a3 and the btn
        
async def Top_5_MF(a1: str,a2: str,a3: str):
    qa_input = QAInput(
    tenure=a1,
    sip=a2,
    fund_expectation=a3
    )
    prompt = f"""
    Tenure: {qa_input.tenure}
    Monthly SIP: {qa_input.sip}
    Expected CAGR: {qa_input.fund_expectation}

    Based on this information, pick the top 5 mutual funds.
    """
    with trace("test1"):
        results=await Runner.run(Mf_list, prompt)
        output = results.final_output
        print(output)

        if not output or not output.top_5:
            return "⚠️ Could not generate mutual fund list. Please try again."

        return "### 📈 Top 5 Mutual Funds\n" + "\n".join(f"{i+1}. {fund}" for i, fund in enumerate(output.top_5))
async def Run_Report(a1:str,a2:str,a3:str):
    selected_input={
        "tenure": a1,
        "sip": a2,
        "fund_expectation":a3
    }
    prompt1 = f"""
    Tenure: {selected_input['tenure']}
    Monthly SIP: {selected_input['sip']}
    Expected CAGR: {selected_input['fund_expectation']}

    Based on this information, pick the best mutual fund and generate a report.
    """
    with trace("test2"):
        
        results1=await Runner.run(mf_selector, prompt1)
        print(results1.final_output)
        return (results1.final_output,gr.update(visible=True))
@function_tool
def send_email(report: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    print(f"in tool{report}")
    subject="Mutual fund report"
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("") # Change this to your verified email
    to_email = To("") # Change this to your email
    content = Content("text/html", report)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return "success"
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You MUST call the send_email tool exactly once.
Do not return text.
Only call the tool, send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)
async def send_email_report(report: str):
    """ Use the email agent to send an email with the report """
    print(f"Writing email...{report}")
    result = await Runner.run(email_agent, report)
    print("Email sent")
    return report
with gr.Blocks(theme=gr.themes.Default(primary_hue="sky"),title="Mutual Fund Selector") as ui:
    gr.Markdown("# Mutual Fund Selector")
    gr.Markdown("Enter your mutual fund selection based query")
    query_textbox = gr.Textbox(label="Your Query", placeholder="Pick a good mutual fund for me to invest in the indian stock market")
    run_button = gr.Button("Get Clarifying Questions", variant="primary")
    c_questions = gr.Markdown(label="Clarifying Questions:")
    a1=gr.Textbox(label="Answer for Question 1",visible=False)
    a2=gr.Textbox(label="Answer for Question 2",visible=False)
    a3=gr.Textbox(label="Answer for Question 3",visible=False)
    #mf_list_btn=gr.Button("Top 5 Mutual funds",variant="primary",visible=False)
    final_report_btn = gr.Button(
    "Generate Final Fund Report",
    variant="primary",
    visible=False
    )

    final_report = gr.Markdown()
    query_textbox.submit(fn=Run_Clarifier, inputs=query_textbox, outputs=[c_questions,a1,a2,a3,final_report_btn])
    run_button.click(fn=Run_Clarifier, inputs=query_textbox, outputs=[c_questions,a1,a2,a3,final_report_btn])
    email_btn=gr.Button("Email the Report",variant="primary",visible=False)
    final_report_btn.click(
    fn=Run_Report,
    inputs=[a1, a2, a3],
    outputs=[final_report,email_btn]
    )
    email_btn.click(fn=send_email_report,inputs=final_report)
    
    

ui.launch(inbrowser=True)
